In [ ]:
#importaciones
from IPython.display import HTML
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import _converter
%matplotlib inline 
plt.style.use('default')
import datetime
import math
sns.set(style="whitegrid") # seteando tipo de grid en seaborn

In [ ]:
# Leyendo csvs
df_vistas = pd.read_csv("../datos_navent_fiuba/fiuba_3_vistas.csv", parse_dates=['timestamp'])
df_vistas.columns = df_vistas.columns.str.lower()

df_postulaciones = pd.read_csv("../datos_navent_fiuba/fiuba_4_postulaciones.csv", parse_dates=['fechapostulacion'])
df_avisos_detalle = pd.read_csv("../datos_navent_fiuba/fiuba_6_avisos_detalle.csv")

In [ ]:
# Elegimos un rango comun entre los dos sets de datos
min_fecha = max(df_vistas['timestamp'].min(), df_postulaciones['fechapostulacion'].min())

df_vistas_gp_aviso = df_vistas.loc[min_fecha <= df_vistas['timestamp']]\
                              .groupby('idaviso')\
                              .agg({'timestamp':'count'})\
                              .reset_index()\
                              .rename(columns={'timestamp': 'vistas_count'})

#vistas_threshold = df_vistas_gp_aviso['vistas_count'].median()
vistas_threshold = 1000

df_posts_gp_aviso = df_postulaciones.loc[min_fecha <= df_postulaciones['fechapostulacion']]\
                                      .groupby('idaviso')\
                                      .agg({'fechapostulacion':'count'})\
                                      .reset_index()\
                                      .rename(columns={'fechapostulacion': 'posts_count'})
df_ef = df_vistas_gp_aviso.merge(df_posts_gp_aviso, on='idaviso')

df_ef = df_ef.loc[df_ef['vistas_count'] > vistas_threshold]

df_ef['ef'] = df_ef['posts_count'] / df_ef['vistas_count']

df_ef.sort_values('ef', ascending=False)